<a href="https://colab.research.google.com/github/jcvancity2022/Computer-Vision-Project-Overview/blob/JeffreyBranch/true_fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News Detection Using DistilBERT
This project fine-tunes a DistilBERT model to classify news articles
as Fake (0) or Real (1). We combined multiple public misinformation
datasets and trained a binary classifier using HuggingFace Transformers.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Setup Python Libraries (pip)

In [2]:
#install some Python packages with pip

!pip install numpy torch datasets transformers evaluate --quiet



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00


In [3]:
# let's check the version we are using

!pip freeze | grep -E '^numpy|^torch|^datasets|^transformers|^evaluate'

datasets==4.0.0
evaluate==0.4.6
numpy==2.0.2
torch==2.9.0+cu126
torchao==0.10.0
torchaudio==2.9.0+cu126
torchdata==0.11.0
torchsummary==1.5.1
torchtune==0.6.1
torchvision==0.24.0+cu126
transformers==4.57.1


# Create IMDB Dataset for Fine-tuning BERT

In [4]:
class_labels = {
    0: "Fake",
    1: "Real"
}


## Let's load the IMDB Dataset

fake-and-real-news-dataset

Fake News Detection

In [ ]:
# ----------------------------------------------------------
# COMBINING BOTH DATASETS PROPERLY (NO DUPLICATES)
# ----------------------------------------------------------

import pandas as pd

# ---------------------------
# Dataset A
# ---------------------------
dfA_fake = pd.read_csv("/content/drive/MyDrive/fake-and-real-news-dataset/Fake.csv")
dfA_true = pd.read_csv("/content/drive/MyDrive/fake-and-real-news-dataset/True.csv")

dfA_fake["label"] = 0   # Fake
dfA_true["label"] = 1   # Real

# Keep consistent columns
dfA_fake = dfA_fake[["title", "text", "subject", "date", "label"]]
dfA_true = dfA_true[["title", "text", "subject", "date", "label"]]

dfA = pd.concat([dfA_fake, dfA_true], axis=0)


# ---------------------------
# Dataset B
# ---------------------------
dfB_fake = pd.read_csv("/content/drive/MyDrive/Fake News Detection/fake.csv")
dfB_true = pd.read_csv("/content/drive/MyDrive/Fake News Detection/true.csv")

dfB_fake["label"] = 0   # Fake
dfB_true["label"] = 1   # Real

# Keep consistent columns
dfB_fake = dfB_fake[["title", "text", "subject", "date", "label"]]
dfB_true = dfB_true[["title", "text", "subject", "date", "label"]]

dfB = pd.concat([dfB_fake, dfB_true], axis=0)


# ---------------------------
# Combine A + B and remove duplicates
# ---------------------------
df_all = pd.concat([dfA, dfB], axis=0).drop_duplicates().reset_index(drop=True)

# Shuffle data randomly
df_all = df_all.sample(frac=1, random_state=42).reset_index(drop=True)

df_all.head()
df_all.shape


## Let's create the train, validation, test sets

In [7]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

# Split into train (70%) and temp (30%)
train_df, temp_df = train_test_split(
    df_all,
    test_size=0.30,
    random_state=42,
    stratify=df_all["label"]
)

# Split temp into val (15%) and test (15%)
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.50,
    random_state=42,
    stratify=temp_df["label"]
)

# Convert to HuggingFace Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Bundle into DatasetDict
dataset = DatasetDict({
    "train": train_dataset,
    "val": val_dataset,
    "test": test_dataset
})

# Remove unwanted auto-generated index column
for split in ["train", "val", "test"]:
    dataset[split] = dataset[split].remove_columns(["__index_level_0__"])

dataset



DatasetDict({
    train: Dataset({
        features: ['title', 'text', 'subject', 'date', 'label'],
        num_rows: 31428
    })
    val: Dataset({
        features: ['title', 'text', 'subject', 'date', 'label'],
        num_rows: 6735
    })
    test: Dataset({
        features: ['title', 'text', 'subject', 'date', 'label'],
        num_rows: 6735
    })
})

## We start by tokenizing our dataset with the BERT's Fast Tokenizer

In [8]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_fast=True)

def tokenize_function(batch):
    return tokenizer(batch["text"], truncation=True, padding=True)

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["title", "text", "subject", "date"]
)

tokenized_dataset


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/31428 [00:00<?, ? examples/s]

Map:   0%|          | 0/6735 [00:00<?, ? examples/s]

KeyboardInterrupt: 

# Setup Training Metrics (Accuracy, F1)

In [ ]:
import evaluate
import numpy as np

# Load accuracy and F1 evaluation modules
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

# Define function that HuggingFace Trainer will call
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert model logits -> predicted class (0 or 1)
    predictions = np.argmax(logits, axis=-1)

    # Compute accuracy and F1 using HuggingFace's evaluate package
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels)

    # Return BOTH metrics
    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"]
    }


# Setup Training Configurations

In [ ]:
# --- Disable Weights & Biases (WandB) COMPLETELY ---
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_SILENT"] = "true"

import wandb
wandb.init = lambda *args, **kwargs: None
wandb.login = lambda *args, **kwargs: None
# ---------------------------------------------------

from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "Fake", 1: "Real"},
    label2id={"Fake": 0, "Real": 1}
)

training_args = TrainingArguments(
    seed=42,
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    compute_metrics=compute_metrics
)




# Fake News Detection Dataset
Fine-Tune DistilBERT on Fake vs Real News Dataset

In [ ]:
# let's fine-tune BERT with the IMDb dataset

trainer.train()

In [ ]:
# let's see how well it did in the test set

trainer.evaluate(tokenized_dataset['test'])

**WOAH!** We got a **92% Accuracy (eval_accuracy)** and **92% F1 (eval_f1)** with just **1 epoch**! 🤯

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

# Run prediction
pred = trainer.predict(tokenized_dataset["test"])
y_true = pred.label_ids
y_pred = np.argmax(pred.predictions, axis=1)

# Build confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Fake", "Real"])

disp.plot(cmap="Blues")
plt.title("Confusion Matrix – Test Set")
plt.show()



# Try out some examples!

In [ ]:
from transformers import pipeline
import torch

device = 0 if torch.cuda.is_available() else -1

news_classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    device=device
)


In [ ]:
fake_news_example = """
BREAKING: Scientists warn that the sun will explode next Friday unless
all citizens donate $50 immediately. Anonymous experts confirm the threat
but provide no scientific evidence. Social media influencers are urging
everyone to act fast before it's too late.
"""
news_classifier(fake_news_example)


In [ ]:
real_news_example = """
The U.S. Department of Labor released updated employment numbers today,
showing moderate job growth in manufacturing and technology sectors.
Economists say the trend aligns with long-term projections for the year.
"""
news_classifier(real_news_example)


In [ ]:
model.save_pretrained("/content/drive/MyDrive/fake-news-final-model/")
tokenizer.save_pretrained("/content/drive/MyDrive/fake-news-final-model/")



# Resources

### If you would like to use this model without running the entire notebook, try the model at my [HuggingFace](https://huggingface.co/wesleyacheng/movie-review-sentiment-classifier-with-bert).

### If you woud like to get this in GitHub, here's my [repo](https://github.com/wesleyacheng/movie-review-sentiment-classifier-with-bert).